In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install pycryptodomex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.4 MB/s eta 0:00:00


In [3]:
from base64 import b64encode, b64decode
import hashlib
from Cryptodome.Cipher import AES
import os
from Cryptodome.Random import get_random_bytes

In [4]:
#definition of all the functions 

def tostring(lst):
    return ','.join(str(x) for x in lst)

def tolist(str):
    return str.split(',')
    
# define encrypt
def encrypt(plain_text, password):
    # generate a random salt
    salt = get_random_bytes(AES.block_size)

    # use the Scrypt KDF to get a private key from the password
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create cipher config
    cipher_config = AES.new(private_key, AES.MODE_GCM)

    # return a dictionary with the encrypted text
    cipher_text, tag = cipher_config.encrypt_and_digest(bytes(plain_text, 'utf-8'))
    return [
        b64encode(cipher_text).decode('utf-8'),
        b64encode(salt).decode('utf-8'),
        b64encode(cipher_config.nonce).decode('utf-8'),
        b64encode(tag).decode('utf-8')
    ]

# define decrypt
def decrypt(enc_list, password):
    # decode the dictionary entries from base64
    salt = b64decode(enc_list[1])
    cipher_text = b64decode(enc_list[0])
    nonce = b64decode(enc_list[2])
    tag = b64decode(enc_list[3])
    

    # generate the private key from the password and salt
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create the cipher config
    cipher = AES.new(private_key, AES.MODE_GCM, nonce=nonce)

    # decrypt the cipher text
    decrypted = cipher.decrypt_and_verify(cipher_text, tag)

    return decrypted



In [5]:
meta = [100, 233, 455, 56, 78, 9, 40, 60]
metastr = tostring(meta)

In [6]:
metastr

'100,233,455,56,78,9,40,60'

In [7]:
#encryption:
#input password, encrypt metadata and obtain text to be hidden 

password = input("Password: ")
encrypted = encrypt(metastr, password)
hide_text = tostring(encrypted)

Password: manohar


In [8]:
encrypted

['+5LMJwlNTGotSTQdmahh6Qk4T8sNLlt8IA==',
 '1h7PyLTzK+k/fPwHAn/wJQ==',
 'xjGIjwcuEVwgg96cPXcXMA==',
 'FIoVlIAvfVRHoX9wKjuMhg==']

In [9]:
hide_text

'+5LMJwlNTGotSTQdmahh6Qk4T8sNLlt8IA==,1h7PyLTzK+k/fPwHAn/wJQ==,xjGIjwcuEVwgg96cPXcXMA==,FIoVlIAvfVRHoX9wKjuMhg=='

In [12]:
#decryption:

#converting the hidden text back to list
encrypted = tolist(hide_text)

#input password
decrypt_password = input('enter password: ')

#decrypt
try:
    decrypted = decrypt(encrypted, decrypt_password)
    metastr_obt = bytes.decode(decrypted)
except:
    print("Incorrect Password!!")
    metastr_obt = '0'


enter password: manohar


In [13]:
# converting back to list

meta_obt = tolist(metastr_obt)
meta_obt = [int(x) for x in meta_obt]

In [14]:
meta_obt

[100, 233, 455, 56, 78, 9, 40, 60]